In [4]:
#do the imports
from __future__ import print_function
import tensorflow as tf

In [22]:
#load csv file
length = 15120 #excluding header
filename = '/tmp/csv_data/train.csv'
'''
content of train.csv
f1,f2,f3,c
1.0,2.0,1.1,1
2.2,1.4,2.3,0
9.4,0.9,5.2,1
....
....
....
'''
filename_queue = tf.train.string_input_producer([filename])
reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)

record_defaults = [[0.0],[0.0],[0.0],[0]]
col1, col2, col3, col4 = tf.decode_csv(value, record_defaults=record_defaults)
features = tf.pack([col1, col2, col3])

print(tf.size(features))

Tensor("Size_1:0", shape=(), dtype=int32)


In [ ]:
#start the session and print the content
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    for i in range(length):
        example, label = sess.run([features, col4])
        print(example, label)
    
    coord.request_stop()
    coord.join(threads)

In [2]:
#load data approach 2, load into python arrays
import numpy as np
filename = '/tmp/csv_data/data.csv'
data_raw = []
n_classes = 2
with open(filename) as f:
    for l in f:
        data_raw.append(l.rstrip())
train_size = int(len(data_raw)*0.8)
test_size = len(data_raw) - train_size
train_raw = data_raw[0:train_size]
test_raw = data_raw[train_size:]

train_x = []
train_y = []
for t in train_raw[1:]:
    parts = t.split(',')
    k = (len(parts) - n_classes)
    train_x.append(np.asarray(parts[:k]))
    train_y.append(np.asarray(parts[k:]))
    
test_x = []
test_y = []
for t in test_raw[1:]:
    parts = t.split(',')
    k = (len(parts) - n_classes)
    test_x.append(np.asarray(parts[:k]))
    test_y.append(np.asarray(parts[k:]))

In [5]:
#build a NN model
#modified from https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py
n_input = 3
n_hidden_1 = 2
training_epochs = 15
n_instances = len(train_x)
batch_size = 100
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with Sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

display_step = 1
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_instances/batch_size)
        # Loop over all batches
        i = 0
        while i < len(train_x):
            start = i
            end = start + batch_size
            batch_x = train_x[start:end]
            batch_y = train_y[start:end]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
            i = end
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: test_x, y: test_y}))

Epoch: 0001 cost= 1.747185184
Epoch: 0002 cost= 1.400322704
Epoch: 0003 cost= 1.117476935
Epoch: 0004 cost= 0.892890539
Epoch: 0005 cost= 0.770114107
Epoch: 0006 cost= 0.710087800
Epoch: 0007 cost= 0.677792409
Epoch: 0008 cost= 0.656868564
Epoch: 0009 cost= 0.643386118
Epoch: 0010 cost= 0.637055476
Epoch: 0011 cost= 0.636220660
Epoch: 0012 cost= 0.634465064
Epoch: 0013 cost= 0.627410255
Epoch: 0014 cost= 0.615845148
Epoch: 0015 cost= 0.601529678
Optimization Finished!
Accuracy: 0.625
